<hr>

Let's consider a single qubit:


In [2]:
from spheres import *

qubit = qt.rand_ket(2)

dirac(qubit)
print("\nxyz: %s\n" % spinor_xyz(qubit))
print(qubit*qubit.dag())

-0.581+0.481i |0> Pr: 0.568
-0.654+-0.060i |1> Pr: 0.432

xyz: [0.70194258 0.69895773 0.13687478]

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.56843739+0.j         0.35097129-0.34947887j]
 [0.35097129+0.34947887j 0.43156261+0.j        ]]


We see here it represented in Dirac braket notation, in terms of its point on the sphere in cartesian coordinates, and its density mtrix $\mid \phi \rangle \langle \phi \mid$. Note that the probabilities appear along the diagonal of this matrix.

What happens during an entangling interaction?

In [4]:
a, b = components(qubit)
entangled = a*bitstring_basis("00") + b*bitstring_basis("11")

dirac(entangled)
print("\nqubit partial state: \nxyz: %s\n%s" % (spinj_xyz(entangled.ptrace(0)), entangled.ptrace(0)))
print("\napparatus partial state: \nxyz: %s\n%s" % (spinj_xyz(entangled.ptrace(1)), entangled.ptrace(1)))

print(qt.entropy_vn(entangled.ptrace(0)))
print(qt.entropy_vn(entangled.ptrace(1)))

-0.581+0.481i |00> Pr: 0.568
-0.654+-0.060i |11> Pr: 0.432

qubit partial state: 
xyz: [0.         0.         0.06843739]
Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.56843739 0.        ]
 [0.         0.43156261]]

apparatus partial state: 
xyz: [0.         0.         0.06843739]
Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.56843739 0.        ]
 [0.         0.43156261]]
0.6837503574022689
0.6837503574022689


What we can see here is the phenomenon of decoherence: if we look at the reduced density matrices, which encode all the information locally accessible at that quantum subsystem, we can see that while they still contain the same probabilities for a $Z$ measurement along the diagonal, the off-diagonal terms have gone to 0, and so the full information about the qubit's (x, y, z) point is no longer localized at the qubit, but rather is contained in the entangled state as a whole. 

In [ ]:
n_experiments = 4

experiment = (sum([qt.tensor(qt.basis(2,0), qt.basis(n, i), qt.basis(n, i)) for i in range(a)]) +\
              sum([qt.tensor(qt.basis(2,1), qt.basis(n, i), qt.basis(n, i)) for i in range(a, n)])).unit()

experiments = qt.tensor(*[experiment]*n_experiments)


In [ ]:
up_outcome = sum([qt.tensor(qt.basis(2,0), qt.basis(n, i), qt.basis(n, i)) for i in range(a)]).unit()
down_outcome = sum([qt.tensor(qt.basis(2,1), qt.basis(n, i), qt.basis(n, i)) for i in range(a, n)]).unit()


In [1]:
from spheres import *

n_env = 10
pure = qt.rand_ket(2)
environment = qt.rand_ket(2**n_env)
environment.dims = [[2]*n_env, [1]*n_env]
g = np.random.randn(n_env)
H = tensor_upgrade(qt.sigmaz(), 0, n_env+1)*sum([g[k-1]*tensor_upgrade(qt.sigmaz(), k, n_env+1) for k in range(1, n_env+1)])
state = qt.tensor(pure, environment)

dirac(pure)
print()
print(state.ptrace(0))

-0.628+-0.686i |0>
0.342+-0.136i |1>

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[ 0.86466356+0.j         -0.121481  -0.31978564j]
 [-0.121481  +0.31978564j  0.13533644+0.j        ]]


In [2]:
qt.commutator(tensor_upgrade(qt.sigmaz(), 0, n_env+1), H).norm()

0.0

In [19]:
dt = 0.001
U = (-1j*H*dt).expm()

from IPython.display import display
import ipywidgets as widgets
out = widgets.HTML()
display(out)

for i in range(10):
    state = U*state
    dm = state.ptrace(0).full()
    out.value = "<br>".join([str(row) for row in dm])

HTML(value='')

KeyboardInterrupt: 

2.0

<hr>

Axioms:

- States of quantum systems are represented by vectors in some Hilbert space.
- Evolutions are unitary, obeying the Schrodinger equation.
- Immediate repetition of a measurement yields the sme outcome.
- Measurement outcome is one of the orthonormal eigenstates of the measured observable.
- Probability given by Born's rule $\mid \langle \phi_{n} | \psi \rangle\mid^2$.
- Universe consists of subystems. Cf. Tensor product.

Different versions of you in each branch. But what is the preferred basis?

Environment induced superselection: "einselection". Constant monitoring of the system by the environment. Picks out special observerable(s) whose states form a pointer basis. E.g., these observables commute with the interaction Hamiltonian: $[O, H_{int}] = 0$.

Consider evolution of the form:

$e^{-it\hat{H}} = \sum_{s} \mid s \rangle \langle s \mid \otimes U_{s}$

Where the first part acts on the system and the second part on the environment. The $U_{s}$ are arbitrary unitaries which govern the evolution of environment conditional on the state of the system. The $\mid s \rangle$ form a pointer basis. Under this evolution, the density matrix of the system will decohere, being diagonal precisely in the $\mid s \rangle$ basis. 

<img src="img/pure_decoherence.png">

The Objective past of a quantum universe: Redundant records of consistent histories

"Con- tinuous monitoring of S by its environment results in re- dundant records of in E. Thus, many observers can find out state of the system indirectly, from small fragments of the same E that caused decoherence. Recent and still ongoing studies discussed in Section IV show how this replication selects “fittest” states that can survive moni- toring, and yield copious information-theoretic offspring: Quantum Darwinism favors pointer observables at the expense of their complements. Objectivity of preferred is quantified by their redundancy – by the number of copies of the state of the system deposited in E. Stabil- ity in spite of the environment is clearly a prerequisite for large redundancy. Hence, pointer states do best in this information - theoretic “survival of the fittest”.

Relative States and the Environment: Einselection, Envariance, Quantum Darwinism, and the Existential Interpretation



Problem of factoring into subsystems: Sean Caroll.



Orthogonality:


A measurement means we start out in a state like:

$$ \mid \psi \rangle\mid 0 \rangle = (\sum_{i}c_{i}\mid s_{i}\rangle)\mid 0 \rangle $$

And unitarily end up in a state like:

$$ \sum_{i}c_{i}\mid s_{i} \rangle\mid e_{i} \rangle $$

The norm must be preserved.



In [205]:
a, b = qt.rand_ket(2), qt.rand_ket(2)
c, d = qt.rand_ket(2), qt.rand_ket(2)

amp = qt.tensor(c, d).dag()*qt.tensor(a, b)
amp2 = c.dag()*a*d.dag()*b

amp, amp2

(Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
 Qobj data =
 [[0.09755705-0.06580976j]],
 Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
 Qobj data =
 [[0.09755705-0.06580976j]])

In [203]:
amp2*amp2

Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.43851354+0.73140366j]]

"Reproducibility of local operations on different sub- systems is also a generic characteristic of quantum field theories (QFT) that is less well-known in the quantum in- formation community. Working in the vacuum state on Minkowski spacetime, the Reeh-Schlieder theorem [16] entails that any operator supported in a local region can be reproduced by another operator in a different local region, potentially at spacelike separations. This result implies that the vacuum is entangled between any set of local regions, and has implications for the localizability of quantum fields [17, 18]."

Operational symmetries of entangled states

In [29]:
from spheres import *

n = 3
a = np.random.randint(1,n)
b = n-a
fine_up = sum(qt.tensor(qt.basis(n, i), qt.basis(n, i)) for i in range(a)).unit()
fine_down = sum(qt.tensor(qt.basis(n, i), qt.basis(n, i)) for i in range(a, n)).unit()

qubit_app_env = (sum([qt.tensor(qt.basis(2,0), qt.basis(n, i), qt.basis(n, i)) for i in range(a)]) +\
                 sum([qt.tensor(qt.basis(2,1), qt.basis(n, i), qt.basis(n, i)) for i in range(a, n)])).unit()

dirac(qubit_app_env)

0.577 |000> Pr: 0.333
0.577 |011> Pr: 0.333
0.577 |122> Pr: 0.333


In [31]:
dirac(fine_up)

0.707 |00> Pr: 0.500
0.707 |11> Pr: 0.500


In [10]:
n_experiments = 3
experiments = qt.tensor(*[qubit_app_env for i in range(n_experiments)])
dirac(experiments)

0.192 |000000000> Pr: 0.037
0.192 |000000011> Pr: 0.037
0.192 |000000122> Pr: 0.037
0.192 |000011000> Pr: 0.037
0.192 |000011011> Pr: 0.037
0.192 |000011122> Pr: 0.037
0.192 |000122000> Pr: 0.037
0.192 |000122011> Pr: 0.037
0.192 |000122122> Pr: 0.037
0.192 |011000000> Pr: 0.037
0.192 |011000011> Pr: 0.037
0.192 |011000122> Pr: 0.037
0.192 |011011000> Pr: 0.037
0.192 |011011011> Pr: 0.037
0.192 |011011122> Pr: 0.037
0.192 |011122000> Pr: 0.037
0.192 |011122011> Pr: 0.037
0.192 |011122122> Pr: 0.037
0.192 |122000000> Pr: 0.037
0.192 |122000011> Pr: 0.037
0.192 |122000122> Pr: 0.037
0.192 |122011000> Pr: 0.037
0.192 |122011011> Pr: 0.037
0.192 |122011122> Pr: 0.037
0.192 |122122000> Pr: 0.037
0.192 |122122011> Pr: 0.037
0.192 |122122122> Pr: 0.037


In [35]:
for i in range(n_experiments):
    perms = list(set(list(permutations([1]*i + [0]*(n_experiments-i)))))
    print(perms)
    for perm in perms:
        sum([qt.tensor(qt.basis(2,int(p)), fine_up if p == 0 else fine_down)])
    print(i)
    dirac(sm)

[(0, 0, 0)]


TypeError: int() argument must be a string, a bytes-like object or a number, not 'tuple'

In [62]:
from spheres import *
from qutip.qip.operations import cnot

state = qt.tensor(qt.rand_ket(2), qt.basis(2,0), qt.basis(2,0))
dirac(state)

-0.708+-0.629i |000> Pr: 0.897
0.147+-0.285i |100> Pr: 0.103


In [63]:
state2 = cnot(N=3, control=0, target=1)*state
dirac(state2)
print(qt.entropy_vn(state2.ptrace(0)))

-0.708+-0.629i |000> Pr: 0.897
0.147+-0.285i |110> Pr: 0.103
0.3318006955577651


In [64]:
state3 = cnot(N=3, control=1, target=2)*state2
dirac(state3)

-0.708+-0.629i |000> Pr: 0.897
0.147+-0.285i |111> Pr: 0.103


In [49]:
state3.ptrace(0)

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.12435798 0.        ]
 [0.         0.87564202]]

In [65]:
qt.entropy_vn(state3.ptrace(2))

0.3318006955577651

In [35]:
np.log(2)

0.6931471805599453